Getting all sentences in text file i.e., train.txt.raw having a specific noun. Noun_list is the list of all nouns having multiple presence in sentences.
1. this will read word-list
2. find all sentences belonging to each word in word-list
3. save each file saperately with the name of word from word list
4. currently, we are focusing only Train file of Gold-PMB

the code below will get all the sentences where noun is used including prefix and postfix. e.g., for arm, it will also give bendarm, and armwrestling etc ...

In [ ]:
# Open and read the file containing the list of words
with open('word_list.txt.raw', 'r') as f:
    word_list = [line.strip() for line in f.readlines()]

# Create a dictionary to store sentences by word
sentences_by_word = {word: [] for word in word_list}

# Open and read the file containing sentences
with open('sentences.txt.raw', 'r') as f:
    sentences = f.readlines()

# Loop through each sentence and check if it contains any of the words
for sentence in sentences:
    for word in word_list:
        if word in sentence:
            sentences_by_word[word].append(sentence.strip())

# Print out the sentences for each word and save them to a file
for word in word_list:
    print(f"Sentences containing '{word}':")
    with open(f"/content/{word}_sentences.txt", "w") as f:
        for sentence in sentences_by_word[word]:
            print(sentence)
            f.write(sentence + "\n")
    print(f"Sentences containing '{word}' have been saved to '{word}_sentences.txt'.\n")


the code below will return only those sentences that have exactly the noun given in the sentence without any prefix or postfix values.

In [ ]:
# Load word list
with open("word_list.txt.raw", "r") as f:
    word_list = [word.strip() for word in f.readlines()]

# Open a file for each word in the word list
word_files = {word: open(f"{word}_sentences.txt", "w") for word in word_list}

# Read the text file line by line and write matching sentences to the corresponding word files
with open("sentences.txt.raw", "r") as f:
    for line in f:
        sentence_words = line.strip().split()
        for word in word_list:
            if word in sentence_words:
                word_files[word].write(line)

# Close all word files
for file in word_files.values():
    file.close()


**code below will move all the text files generated through above code into sentences folder.**

In [ ]:
import os
import shutil

# Define the name of the directory where we want to move the files
directory_name = "sentences"

# Create the directory if it doesn't exist
if not os.path.exists(directory_name):
    os.mkdir(directory_name)

# Get a list of all the files in the current directory
files = os.listdir()

# Loop through the files and move any text files to the 'sentences' directory
for file in files:
    if file.endswith(".txt"):
        source_path = os.path.abspath(file)
        destination_path = os.path.join(os.getcwd(), directory_name, file)
        shutil.move(source_path, destination_path)
        print(f"Moved {file} to {directory_name}")


**implementing supersense tagging code to each text file to get the supersense for each noun used in multiple sentences.**

In [ ]:
!pip install booknlp
!python -m spacy download en_core_web_sm

In [ ]:
from booknlp.booknlp import BookNLP

# model_params={
# 		"pipeline":"entity,quote,supersense,event,coref",
# 		"model":"big"
# 	}

######## i am only interested in SuperSenses model parameters so i am using only SuperSense.
model_params={
		"pipeline":"entity,supersense",
		"model":"big"
	}


booknlp=BookNLP("en", model_params)


**code below will deal with 1 file only.**

In [ ]:

# Input file to process
input_file="CD_sentences.txt"

# Output directory to store resulting files in
output_directory="output_dir/bartleby/"

# File within this directory will be named ${book_id}.entities, ${book_id}.tokens, etc.
book_id="bartleby"

booknlp.process(input_file, output_directory, book_id)

**code to read all files one by one and save the processed data to each saperate folder.**

In [ ]:
import os
#import booknlp

# set the directory where the text files are located
input_directory = '/content/sentences/'

# loop through all files in the input directory
for input_file in os.listdir(input_directory):
    # check if the file has the .txt extension
    if input_file.endswith('.txt'):
        # create an output directory for the file
        output_directory = os.path.join('/content/sst_outputs', os.path.splitext(input_file)[0])
        os.makedirs(output_directory, exist_ok=True)
        # File within this directory will be named ${book_id}.entities, ${book_id}.tokens, etc.
        book_id = os.path.splitext(input_file)[0]
        # process the input file and save the output to the output directory
        booknlp.process(os.path.join(input_directory, input_file), output_directory, book_id)


this code will download the 2 folder

In [ ]:
import shutil
import os

# compress the two folders into a zip archive
shutil.make_archive("folders", "zip", "/content/sst_outputs", "/content/sentences")

# download the zip archive to your local machine
from google.colab import files
files.download("folders.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**splitting dataset files into sub-filels**

splitting text files

In [ ]:
def chunks(l, n):
    """Chunks iterable into n sized chunks"""
    for i in range(0, len(l), n):
        yield l[i:i + n]

# Collect all lines, without loading whole file into memory
lines = []
with open('dev.txt.raw') as main_file:
    for line in main_file:
        lines.append(line)

# Write each group of lines to separate files
for i, group in enumerate(chunks(lines, n=1), start=1):
    with open('File%d.txt' % i, mode="w") as out_file:
        for line in group:
            out_file.write(line)

moving files to folder

In [ ]:
import os
import shutil

# Define the name of the directory where we want to move the files
directory_name = "text-files"

# Create the directory if it doesn't exist
if not os.path.exists(directory_name):
    os.mkdir(directory_name)

# Get a list of all the files in the current directory
files = os.listdir()

# Loop through the files and move any text files to the 'sentences' directory
for file in files:
    if file.endswith(".txt"):
        source_path = os.path.abspath(file)
        destination_path = os.path.join(os.getcwd(), directory_name, file)
        shutil.move(source_path, destination_path)
        print(f"Moved {file} to {directory_name}")


**to varify number of files in each folder. we count files on both folders.**

In [ ]:
import os

# folder path
dir_path = r'/content/text-files'
count = 0
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('File count:', count)

File count: 885


splitting DRS files

In [ ]:
def chunk_file(name, lines_per_chunk, chunks_per_file):

    def write_chunk(chunk_no, chunk):
        with open("file{}.txt".format(chunk_no), "w") as outfile:
            outfile.write("".join(i for i in chunk))

    count, chunk_no, chunk_count, chunk = 1, 1, 0, []
    with open(name, "r") as f:
        for row in f:
            if count > lines_per_chunk and row == "\n":
                chunk_count += 1
                count = 1
                chunk.append("\n")
                if chunk_count == chunks_per_file:
                    write_chunk(chunk_no, chunk)
                    chunk = []
                    chunk_count = 0
                    chunk_no += 1
            else:
                count += 1
                chunk.append(row)
    if chunk:
        write_chunk(chunk_no, chunk)

chunk_file("dev.txt", 3, 1)

**moving all text files into "folder-name" folder**

In [ ]:
import os
import shutil

# Define the name of the directory where we want to move the files
directory_name = "DRS-files"

# Create the directory if it doesn't exist
if not os.path.exists(directory_name):
    os.mkdir(directory_name)

# Get a list of all the files in the current directory
files = os.listdir()

# Loop through the files and move any text files to the 'sentences' directory
for file in files:
    if file.endswith(".txt"):
        source_path = os.path.abspath(file)
        destination_path = os.path.join(os.getcwd(), directory_name, file)
        shutil.move(source_path, destination_path)
        print(f"Moved {file} to {directory_name}")


In [ ]:
mv ./DRS-files/dev.txt .

counting files in the folder

In [ ]:
import os

# folder path
dir_path = r'/content/DRS-files'
count = 0
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('File count:', count)

File count: 885


reading files one by one

In [ ]:
!pip install spacy

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
# Import the required libraries
import os

# Define the location of the directory
path =r"/content/text-files/"

# Change the directory
os.chdir(path)

def read_files(file_path):
   with open(file_path, 'r') as file:
      content = file.read()
      document = nlp(content)
      for token in document:
        if token.pos_ == "NOUN":
          print(token, token.pos_)
      print(content)

# Iterate over all the files in the directory
for file in os.listdir():
   if file.endswith('.txt'):
      # Create the filepath of particular file
      file_path =f"{path}/{file}"

read_files(file_path)

news NOUN
That news got around.



In [ ]:
import glob
import errno

path = '/content/text-files/*.txt'
files = glob.glob(path)
for name in files:
    try:
        with open(name) as f:
          content = open(name)
          print(content)
            #pass # do what you want
    except IOError as exc:
        if exc.errno != errno.EISDIR:
            raise

**Reading both DRS and Text files at a time and processing them.**

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
!pip install booknlp

**Splitting Text Files**

In [ ]:
import os

# Create a folder to store the sub-files
folder_name = 'subfiles'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Open the input file for reading
with open('translation.txt', 'r') as input_file:

    # Read the contents of the input file
    contents = input_file.read()

    # Split the contents based on newlines
    subfiles = contents.split('\n')

    # Loop over the subfiles and save them in the folder
    for i, subfile in enumerate(subfiles):
        if subfile:
            # Create the path to the output file
            output_file_path = os.path.join(folder_name, f'file_{i}.txt')

            # Open a new file for writing
            with open(output_file_path, 'w') as output_file:
                # Write the contents of the subfile to the output file
                output_file.write(subfile)


**splitting DRS files**

In [ ]:
import os

# Create a folder to store the sub-files
folder_name = 'drs_files'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Open the input file for reading
with open('drs.txt', 'r') as input_file:

    # Read the contents of the input file
    contents = input_file.read()

    # Split the contents based on newlines
    subfiles = contents.split('\n\n')

    # Loop over the subfiles and save them in the folder
    for i, subfile in enumerate(subfiles):
        if subfile:
            # Create the path to the output file
            output_file_path = os.path.join(folder_name, f'file_{i}.txt')

            # Open a new file for writing
            with open(output_file_path, 'w') as output_file:
                # Write the contents of the subfile to the output file
                output_file.write(subfile)


**modifying text folder with noun_tags**

In [ ]:
import os
import spacy

# Load the spacy model
nlp = spacy.load('en_core_web_sm')

# Create a folder to store the modified sub-files
output_folder_name = 'text_files_modified'
if not os.path.exists(output_folder_name):
    os.makedirs(output_folder_name)

# Loop over the files in the subfiles folder
input_folder_name = 'text_files'
for input_file_name in os.listdir(input_folder_name):
    input_file_path = os.path.join(input_folder_name, input_file_name)

    # Open the input file for reading
    with open(input_file_path, 'r') as input_file:
        # Read the contents of the input file
        contents = input_file.read()

        # Tokenize the contents using spacy
        doc = nlp(contents)

        # Replace all nouns with noun_tag
        new_tokens = []
        for token in doc:
            if token.pos_ == 'NOUN':
                new_tokens.append(nlp.make_doc('noun_tag')[0])
            else:
                new_tokens.append(token)

        # Create a new doc with the modified tokens
        modified_doc = spacy.tokens.Doc(doc.vocab, words=[token.text for token in new_tokens])

        # Get the modified contents
        modified_contents = modified_doc.text

    # Create the path to the output file
    output_file_path = os.path.join(output_folder_name, input_file_name)

    # Open a new file for writing
    with open(output_file_path, 'w') as output_file:
        # Write the modified contents to the output file
        output_file.write(modified_contents)


**reading nouns for each text file and saving them in another folder's files.**

In [ ]:
import os
import spacy

# Load the spacy model
nlp = spacy.load('en_core_web_sm')

# Define the input and output folder paths
input_folder_path = "text_files"
output_folder_path = "noun_files"

# Make the output folder if it doesn't already exist
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

# Loop over the files in the input folder
for filename in os.listdir(input_folder_path):
    # Get the full path of the input file
    input_file_path = os.path.join(input_folder_path, filename)

    # Read the contents of the input file
    with open(input_file_path, 'r') as f:
        contents = f.read()

    # Process the contents of the input file
    doc = nlp(contents)
    nouns = [token.text for token in doc if token.pos_ == 'NOUN']

    # Create the output file path
    output_file_path = os.path.join(output_folder_path, f'{filename}')

    # Write the nouns to the output file
    with open(output_file_path, 'w') as f:
        f.write('\n'.join(nouns))


**modifying DRS files**

In [ ]:
import os

# Define the paths to the two folders
folder1_path = "noun_files"
folder2_path = "drs_files"
output_path = "drs_files_modified"

# Create the output directory if it doesn't exist
if not os.path.exists(output_path):
    os.mkdir(output_path)

# Iterate over all the files in folder 1
for file1_name in os.listdir(folder1_path):
    # Check if the file is a text file
    if file1_name.endswith(".txt"):
        # Get the full path to the file
        file1_path = os.path.join(folder1_path, file1_name)

        # Read the contents of file 1
        with open(file1_path, 'r') as f:
            file1_contents = f.read()

        # Get the corresponding file in folder 2
        file2_name = file1_name
        file2_path = os.path.join(folder2_path, file2_name)

        # Read the contents of file 2
        with open(file2_path, 'r') as f:
            file2_contents = f.read()

        # Split the contents of file 1 into a list of words
        words_to_replace = file1_contents.split('\n')

        # Loop over the words to replace
        for word in words_to_replace:
            # Replace the word in file 2 if it's present
            file2_contents = file2_contents.replace(word, 'noun_tag')

        # Write the modified contents of file 2 to the output folder
        output_file_path = os.path.join(output_path, file2_name)
        with open(output_file_path, 'w') as f:
            f.write(file2_contents)


**Merging all dataset files into 1 file for both text and DRS.**


merging text files into 1 file.

In [ ]:
import os

# Replace with the appropriate folder path
folder_path = '/content/text_files_modified'

# Replace with the appropriate output file path
output_path = 'modified_text.txt'

with open(output_path, 'w') as outfile:
    for filename in os.listdir(folder_path):
        filepath = os.path.join(folder_path, filename)
        with open(filepath, 'r') as infile:
            outfile.write(infile.read())
            outfile.write('\n')  # Add a newline character between each file


Merging DRS files into 1 file.

In [ ]:
import os

# Replace with the appropriate folder path
folder_path = '/content/drs_files_modified'

# Replace with the appropriate output file path
output_path = 'modified_drs.txt'

with open(output_path, 'w') as outfile:
    for filename in os.listdir(folder_path):
        filepath = os.path.join(folder_path, filename)
        with open(filepath, 'r') as infile:
            outfile.write(infile.read())
            outfile.write('\n\n')  # Add a newline character between each file


**Trying SST using BookNLP**

In [ ]:
!pip install booknlp
!python -m spacy download en_core_web_sm

In [ ]:
from booknlp.booknlp import BookNLP

# model_params={
# 		"pipeline":"entity,quote,supersense,event,coref",
# 		"model":"big"
# 	}

######## i am only interested in SuperSenses model parameters so i am using only SuperSense.
model_params={
		"pipeline":"entity,supersense",
		"model":"big"
	}


booknlp=BookNLP("en", model_params)


In [ ]:

# Input file to process
input_file="text.txt"

# Output directory to store resulting files in
output_directory="text_output/"

# File within this directory will be named ${book_id}.entities, ${book_id}.tokens, etc.
book_id="bartleby"

booknlp.process(input_file, output_directory, book_id)

--- spacy: 0.032 seconds ---
--- entities: 0.504 seconds ---
--- quotes: 0.000 seconds ---
--- name coref: 0.000 seconds ---
--- TOTAL (excl. startup): 4.828 seconds ---, 61 words


applying BookNLP on multiple text files.

In [ ]:
import os
#import booknlp

# Define the input directory containing the text files to process
input_directory = "/content/dev_subfiles/"

# Define the output directory to store the resulting files
output_directory = "sst/"

book_id="bartleby"

# Loop over each file in the input directory
for filename in os.listdir(input_directory):
    # Check that the file is a text file
    if filename.endswith(".txt"):
        # Define the input file path
        input_file = os.path.join(input_directory, filename)
        # Define the book ID based on the filename (without the .txt extension)
        book_id = os.path.splitext(filename)[0]
        # Define the output directory for this book
        book_output_directory = os.path.join(output_directory, book_id)
        # Create the output directory if it doesn't exist
        if not os.path.exists(book_output_directory):
            os.makedirs(book_output_directory)
        # Call the booknlp.process() function to process the input file and save the resulting files
        booknlp.process(input_file, book_output_directory, book_id)


working on small data examples, 10 only.

In [ ]:
import pandas as pd

# Load the supersense file into a pandas DataFrame
supersense_file = 'text_output/bartleby.supersense'
df = pd.read_csv(supersense_file, sep='\t', usecols=[0, 1, 2])
df.columns = ['start_token', 'end_token', 'supersense_category']


In [ ]:
# Load the original text file into a string variable
with open('text.txt', 'r') as f:
    text = f.read()


In [ ]:
import pandas as pd

# Load the supersense file into a pandas DataFrame
supersense_file = 'text_output/bartleby.supersense'
df = pd.read_csv(supersense_file, sep='\t', usecols=[0, 1, 2, 3])
df.columns = ['start_token', 'end_token', 'supersense_category', 'text']

# Filter the DataFrame to only include rows where the supersense category starts with "noun"
noun_df = df[df['supersense_category'].str.startswith('noun')]

# Write the filtered DataFrame to a new file
noun_df.to_csv('noun_supersenses.csv', sep='\t', index=False)


transforming noun.tag --> noun_tag

In [ ]:
import pandas as pd

# Read the "noun_supersenses.csv" file into a pandas DataFrame
df = pd.read_csv('noun_supersenses.csv', sep='\t')

# Replace any "." characters in the "supersense_category" column with "_" characters
df['supersense_category'] = df['supersense_category'].str.replace('.', '_')

# Save the modified DataFrame back to the "noun_supersenses.csv" file
df.to_csv('noun_supersenses.csv', sep='\t', index=False)


<ipython-input-26-3871059852cd>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['supersense_category'] = df['supersense_category'].str.replace('.', '_')


this is based on word matching.

In [ ]:
import csv

# Load the CSV file into a dictionary
with open('noun_supersenses.csv', 'r') as f:
    reader = csv.DictReader(f, delimiter='\t')
    mapping = {row['text']: row['supersense_category'] for row in reader}

# Load the text file and replace matching text with supersense category
with open('text.txt', 'r') as f:
    text = f.read()
    for key, value in mapping.items():
        text = text.replace(key, value)

# Write the modified text back to the file
with open('modified_text.txt', 'w') as f:
    f.write(text)


In [ ]:
ls

drs.txt             modified_text.txt     text_output/  text.txt.raw
modified_text2.txt  noun_supersenses.csv  text.txt
